<a href="https://colab.research.google.com/github/dleegithub/sample_vfi/blob/main/501_FL2022_codingtutorial_unemployed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Unemployed worker's problem
(sample code for the distributed problem set, FL2022 501b) 

In [ ]:
# Import libraries

import numpy as np
# import matplotlib.pyplot as plt
import timeit

In [ ]:
# Speificy parameter values
beta = 0.9
gamma = 0.0
b0 = 0.5 #employment benefit
b1 = 0.8


In [ ]:
### Exogenous Wage Distribution
n = 3_000
w_min, w_max = 1, 2
delta = 0.2

q_default = np.ones(n+1,dtype=float) / (n+1)      # Uniform distribution  

w_default = np.linspace(w_min, w_max, n+1)
w_MPS = np.linspace(w_min-delta, w_max+delta, n+1)

In [ ]:
### Utility Functions
def u(x):
    # return x
    return (x**(1-gamma) - 1.0)/(1-gamma) 

def u_inv(y): # Inverse of utility
    # return y
    return (y*(1-gamma) + 1)**(1/(1-gamma)) 

In [ ]:
### State-action values
def state_action_values(i, v, w=w_default, q=q_default, b=b0):
    # Evaluate value for each state-action pair for i_th grid of wage, given value function v
    # action is either accept or reject the current offer
    Expect = np.sum(v * q)
    accept = u(w[i]) / (1 - beta) 
    reject = u(b) + beta * Expect

    return np.array([accept, reject])

In [ ]:
def compute_reservation_wage(
                             w=w_default,
                             q=q_default,
                             b=b0,
                             max_iter=500,
                             tol=1e-6):
  
    n = len(w)
    v = u(w) / (1 - beta)        # initial guess
    v_next = np.empty_like(v)
    i = 0
    error = tol + 1
    while i < max_iter and error > tol:

        for iw in range(n):
            v_next[iw] = np.max(state_action_values(iw, v, w=w, q=q, b=b))

        error = np.max(np.abs(v_next - v))
        i += 1

        v[:] = v_next  # copy contents into v

    # reservation wage
    expect = np.sum(v * q)
    what = u_inv((1-beta) * (u(b) + beta * expect) )
    return what

In [ ]:
print("Reservation wage is {:.3f} when benefit is {:.1f}".format(compute_reservation_wage(b=b0), b0))

Reservation wage is 1.523 when benefit is 0.5


In [ ]:
print("Reservation wage is {:.3f} when benefit is {:.1f}".format(compute_reservation_wage(b=b1), b1))

Reservation wage is 1.583 when benefit is 0.8


In [ ]:
print("Reservation wage is {:.3f} after an MPS in the wage distribution".format(compute_reservation_wage(b=b0, w=w_MPS)))

Reservation wage is 1.612 after an MPS in the wage distribution


In [ ]:
# %%timeit -r 3 -n 100
# result = compute_reservation_wage(b=b0)

Business Cycle

In [ ]:
pbr = 0.2
prb = 0.2
Gamma_default = np.array([[1-pbr, pbr],[prb, 1-prb]])#state 0: boom, state1: recession
Gamma_eq = np.tile([0.5]*2, (2,1))


In [ ]:
Gamma_default

array([[0.8, 0.2],
       [0.2, 0.8]])

In [ ]:
def state_action_values_cycle(iw, v, Gamma, w, q, b):
    # Evaluate value for each state-action pair for iw_th grid of wage, at each state s, given value function v
    # action is either accept or reject the current offer
    n_s = Gamma.shape[0]
    Expect = np.zeros(n_s,dtype=float)
    for s in range(n_s):
      Expect[s] = np.sum( v[s] * q[s] )
    accept = np.tile( u(w[iw]) / (1 - beta), (n_s,1))
    reject = u(b) + beta *  np.matmul (Gamma ,np.reshape(Expect,(n_s,1))) 
    return np.concatenate((accept, reject), axis=1)

In [ ]:
def compute_reservation_wage_cycle(
                                  w=w_default,
                                  Gamma = Gamma_default,
                                  b=b0,
                                  max_iter=500,
                                  tol=1e-6):
  
    n = len(w)
    n_s = Gamma.shape[0]


    fb = np.reshape(2*(w-1),(1,n)) #density_boom
    q_cycle = np.concatenate((fb / np.sum(fb), 1 / n * np.ones((1,n),dtype=float)), axis=0)

    q = q_cycle

    v = np.tile(u(w) / (1 - beta) , (n_s,1))      # initial guess, v is now (n_s * n_w)
    v_next = np.empty_like(v)
    i = 0
    error = tol + 1
    while i < max_iter and error > tol:

        for iw in range(n):
          v_next[:,iw] = np.max(state_action_values_cycle(iw, v, Gamma=Gamma, w=w, q=q, b=b), axis=1)

        error = np.max(np.abs(v_next - v))
        i += 1

        v[:] = v_next  # copy contents into v

    # reservation wage
    expect = np.zeros(n_s,dtype=float)
    for s in range(n_s):
      expect[s] = np.sum( v[s] * q[s] )

    what = u_inv( (1-beta)  * (u(b) + beta *  np.matmul (Gamma ,np.reshape(expect,(n_s,1))) ) ) # reservation wage using formula
    
    # policy = np.tile(u(w) / (1 - beta) , (n_s,1)) >= u(b) + beta * np.matmul (Gamma ,np.reshape(expect,(n_s,1))) # reservation wage using policy function
    # what = w[np.argmax(policy, axis=1)]
    return what.squeeze()


In [ ]:
what0 = compute_reservation_wage_cycle()
what1 = compute_reservation_wage_cycle(Gamma=Gamma_eq)

In [ ]:
print("Reservation wage is {:.3f} in boom, {:.3f} in recession".format(*what0))
print("Reservation wage is {:.3f} in boom, {:.3f} in recession under equal transition probabilities".format(*what1))

Reservation wage is 1.599 in boom, 1.553 in recession
Reservation wage is 1.580 in boom, 1.580 in recession under equal transition probabilities
